# Importation

In [204]:
#céer une connection avec google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
!pip install -q nltk sentence-transformers unidecode

In [206]:
# Packages
import os, re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize

import nltk
from unidecode import unidecode

In [207]:
# Emplacements des fichiers (fourni par l'utilisateur)
fc_data = pd.read_csv("/content/drive/MyDrive/senscritique/fightclub_critiques.csv")
is_data = pd.read_csv("/content/drive/MyDrive/senscritique/interstellar_critiques.csv")

# Apperçu

In [208]:
# Afficher les dimensions de la table des données (propriété shape)
print(" - Fight Club :", fc_data.shape)
print(" - Interstellar :", is_data.shape)

 - Fight Club : (1000, 11)
 - Interstellar : (1000, 11)


In [209]:
# Afficher la liste des colonnes data.columns
print(" - Fight Club :", fc_data.columns.tolist())
print(" - Interstellar :", is_data.columns.tolist())

 - Fight Club : ['id', 'URL', 'rating', 'review_date_creation', 'review_date_last_update', 'review_hits', 'review_content', 'review_title', 'gen_review_like_count', 'username', 'user_id']
 - Interstellar : ['id', 'URL', 'rating', 'review_date_creation', 'review_date_last_update', 'review_hits', 'review_content', 'review_title', 'gen_review_like_count', 'username', 'user_id']


In [210]:
# Afficher la liste des colonnes et leurs types (propriété dtypes)
print(" - Fight Club :")
print(fc_data.dtypes)
print("\n")
print(" - Interstellar :")
print(is_data.dtypes)

 - Fight Club :
id                          int64
URL                        object
rating                      int64
review_date_creation       object
review_date_last_update    object
review_hits                 int64
review_content             object
review_title               object
gen_review_like_count       int64
username                   object
user_id                     int64
dtype: object


 - Interstellar :
id                          int64
URL                        object
rating                      int64
review_date_creation       object
review_date_last_update    object
review_hits                 int64
review_content             object
review_title               object
gen_review_like_count       int64
username                   object
user_id                     int64
dtype: object


In [211]:
# Afficher les 5 premières lignes fc_data
fc_data.head(5)

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721


In [212]:
# Afficher les 5 premières lignes is_data
is_data.head(5)

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"<p>Chers lecteurs, chères lectrices,</p>\n\n<p...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,<p>Malgré ce que j'entends dire ou lis sur le ...,Tous les chemins mènent à l'Homme,347,blig,133793
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"<p>Un grand film, pour moi, c'est un film qui ...",Rage against the dying of the light.,453,Samu-L,41951
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,<p>Tout en n'étant absolument pas un admirateu...,Inspacetion,273,Nushku,24046


# Pretraitement (80% IA)

In [213]:
# Ajout colonne 'film'
df_fc = fc_data.copy()
df_fc["film"] = "Fight Club"

df_is = is_data.copy()
df_is["film"] = "Interstellar"

In [214]:
# Télécharger stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

print("Stopwords téléchargés. Exemple (fr) :", stopwords.words('french')[:20])
print("Nombre de stopwords FR :", len(stopwords.words('french')))

Stopwords téléchargés. Exemple (fr) : ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur']
Nombre de stopwords FR : 157


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [215]:
# Fusion des deux datasets + ajouter un ID
df = pd.concat([df_fc, df_is], ignore_index=True).reset_index(drop=True)
df["internal_id"] = df.index.astype(int)

# Heuristique simple pour détecter FR / EN (pour stopwords)
text_sample = " ".join(df["review_content"].astype(str).head(50).tolist()).lower()
french_cues = [" le ", " la ", " et ", " pas ", "que ", "je ", "nous "]
is_french = sum(1 for s in french_cues if s in text_sample) >= 2

if is_french:
    sw = set(stopwords.words("french"))
else:
    sw = set(stopwords.words("english"))

print("Total critiques :", len(df))
print("Langue détectée (heuristique) :", "FR" if is_french else "EN/Autre")
print("Stopwords chargés :", len(sw))

print("\nDistribution par film :")
print(df["film"].value_counts())

Total critiques : 2000
Langue détectée (heuristique) : FR
Stopwords chargés : 157

Distribution par film :
film
Fight Club      1000
Interstellar    1000
Name: count, dtype: int64


In [216]:
# Prétraitement du texte
def clean_text(text, sw):
    text = str(text).lower()
    text = unidecode(text)  # enlever accents
    text = re.sub(r"[^a-z0-9\s]", " ", text)  # garder lettres/nombres
    tokens = [w for w in text.split() if w not in sw and len(w) > 2]
    return " ".join(tokens)

# Application au titre + contenu
df["clean_text"] = (df["review_title"].fillna("") + " " + df["review_content"].fillna("")).apply(lambda x: clean_text(x, sw))

# Supprimer les lignes vides ou NaN
df = df.dropna(subset=["review_title", "review_content"]).reset_index(drop=True)

print("Exemple de critique nettoyée :\n")
print("AVANT :", df.loc[0, "review_content"][:200])
print("APRES :", df.loc[0, "clean_text"][:200])
print("\n")
print("Taille du dataset après nettoyage :", df.shape)

Exemple de critique nettoyée :

AVANT : Fight Club sort à la fin 1999 sur les écrans. Le film fut très controversé à sa sortie en salle, et connu un succès pour le moins mitigé, la faute à un marketing mal orchestré et à une critique assez 
APRES : analyse film livre fight club sort fin 1999 ecrans film tres controverse sortie salle connu succes moins mitige faute marketing mal orchestre critique assez frileuse cependant bouche oreille fit sorti


Taille du dataset après nettoyage : (1481, 14)


# Modèle (100% IA)

In [217]:
# Représentation TF-IDF
tfidf = TfidfVectorizer(max_features=5000)  # limiter vocabulaire à 5000 mots les + utiles
X = tfidf.fit_transform(df["clean_text"])

print("Matrice TF-IDF :", X.shape)
print("Nombre de mots distincts gardés :", len(tfidf.get_feature_names_out()))

Matrice TF-IDF : (1481, 5000)
Nombre de mots distincts gardés : 5000


In [218]:
# Calcul des similarités
cosine_sim = linear_kernel(X, X)  # produit scalaire normalisé
print("Matrice de similarité :", cosine_sim.shape)

# Exemple : afficher les 5 critiques les + proches de la 1ère critique
idx = 0
similarities = cosine_sim[idx]
top_indices = similarities.argsort()[::-1][1:6]  # exclure soi-même
print("\nCritique de départ :", df.loc[idx, "review_title"])
print("\nTop 5 critiques similaires :")
for i in top_indices:
    print(f"- {df.loc[i, 'review_title']} ({df.loc[i, 'film']})")

Matrice de similarité : (1481, 1481)

Critique de départ : Analyse du film et du livre

Top 5 critiques similaires :
- CRITIQUE et ANALYSE personnelles dans le détail du début jusqu'à la fin du film. Attention Spoilers! (Fight Club)
- Une Analyse plus qu'une critique...  (Fight Club)
- L'homme que tu peux adorer, puis détester après. Qui est-ce ? (Fight Club)
- Fight Club de Poulets (Fight Club)
- First rule of the Moï club : do not talk about fight club. (Fight Club)


In [219]:
# Fonction de recommandation (même film seulement)
def recommend(review_id, top_n=5):
    film = df.loc[review_id, "film"]
    similarities = cosine_sim[review_id]

    # Garder seulement les critiques du même film
    mask = (df["film"] == film).values

    # Trier les indices par similarité
    indices = similarities.argsort()[::-1]

    # Exclure la critique elle-même
    recommendations = [i for i in indices if i != review_id and mask[i]]

    # Top N
    top_recs = recommendations[:top_n]

    print(f"\nCritique de départ ({film}): {df.loc[review_id, 'review_title']}")
    print("→ Suggestions similaires :\n")
    for i in top_recs:
        print(f"- {df.loc[i, 'review_title']}")

# Exemple : recommander à partir de la critique n°10
recommend(10, top_n=5)


Critique de départ (Fight Club): "I know this because Tyler knows this."
→ Suggestions similaires :

- First Rule
- " la première règle du Fight Club, ne jamais parler du Fight Club"
- Je suis la critique inutile de Jack
- Faut pas le prendre pour ce qu'il n'est pas
-  Je me suis fait passer un savon ! 


In [220]:
from math import log, exp
from datetime import datetime

# --- Helper functions ---
def safe_column(df, col, default=0):
    return df[col] if col in df.columns else pd.Series([default]*len(df), index=df.index)

# Normalisation helpers
def normalize_series_minmax(s):
    if s.max() == s.min():
        return np.zeros_like(s, dtype=float)
    return (s - s.min()) / (s.max() - s.min())

def recency_score(date_series, decay_lambda=0.001):
    now = pd.Timestamp.now()
    days = (now - pd.to_datetime(date_series, errors='coerce')).dt.days.clip(lower=0).fillna(3650)
    return pd.Series(np.exp(-decay_lambda * days), index=date_series.index)


# --- Prepare signals from df ---
# Ensure columns exist, otherwise fill defaults
df['review_hits'] = safe_column(df, 'review_hits', 0).astype(float)
df['rating'] = safe_column(df, 'rating', df.get('score', pd.Series([0]*len(df)))).astype(float)
if 'review_date_creation' in df.columns:
    df['review_date'] = pd.to_datetime(df['review_date_creation'], errors='coerce')
else:
    df['review_date'] = pd.Timestamp.now()

# Precompute normalized popularity/recency/rating
pop_norm_all = pd.Series(normalize_series_minmax(np.log1p(df['review_hits'].fillna(0).astype(float))), index=df.index)
recency_norm_all = pd.Series(normalize_series_minmax(recency_score(df['review_date'], decay_lambda=0.003)), index=df.index) # ajuste lambda
rating_norm_all = pd.Series(normalize_series_minmax(df['rating'].fillna(df['rating'].median())), index=df.index)

# Example: function to compute final score for candidates of a specific review
def compute_combined_scores_for_review(review_idx, cosine_sim_matrix, df,
                                       w_text=0.6, w_pop=0.2, w_rec=0.15, w_rating=0.05):
    """
    review_idx : internal index in df (row number)
    cosine_sim_matrix : full matrix (N x N)
    returns DataFrame with candidate ids and final_score
    """
    film = df.loc[review_idx, 'film']
    # mask same film excluding self
    mask = (df['film'] == film).values
    mask[review_idx] = False

    candidate_idxs = np.where(mask)[0]
    if len(candidate_idxs) == 0:
        return pd.DataFrame(columns=['candidate_idx','final_score'])

    text_sim = cosine_sim_matrix[review_idx, candidate_idxs]  # cosine in [-1,1]
    text_sim_norm = (text_sim + 1.0) / 2.0  # map to [0,1]

    # Ensure these are Series before using .loc
    pop_norm_all_series = pd.Series(pop_norm_all, index=df.index)
    recency_norm_all_series = pd.Series(recency_norm_all, index=df.index)
    rating_norm_all_series = pd.Series(rating_norm_all, index=df.index)

    # Use the indices to select the correct values from the normalized series
    pop_norm = pop_norm_all.loc[candidate_idxs].values
    rec_norm = recency_norm_all.loc[candidate_idxs].values
    rate_norm = rating_norm_all.loc[candidate_idxs].values


    final_score = w_text * text_sim_norm + w_pop * pop_norm + w_rec * rec_norm + w_rating * rate_norm

    out = pd.DataFrame({
        'candidate_idx': candidate_idxs,
        'text_sim': text_sim,
        'text_sim_norm': text_sim_norm,
        'pop_norm': pop_norm,
        'recency_norm': rec_norm,
        'rating_norm': rate_norm,
        'final_score': final_score
    })
    out = out.sort_values('final_score', ascending=False).reset_index(drop=True)
    return out

# --- MMR re-ranking ---
def mmr_rerank(query_idx, candidates_df, cosine_sim_matrix, top_k=5, mmr_lambda=0.7):
    """
    candidates_df must contain 'candidate_idx' and 'final_score' (sorted by final_score desc)
    """
    selected = []
    candidate_idxs = candidates_df['candidate_idx'].tolist()
    # Precompute text similarity matrix between candidates (if needed)
    cand_pos = {cid:i for i,cid in enumerate(candidate_idxs)}
    # Use the relevance (final_score) as base
    while len(selected) < top_k and candidate_idxs:
        mmr_scores = []
        for cid in candidate_idxs:
            rel = float(candidates_df.loc[candidates_df['candidate_idx']==cid, 'final_score'].iloc[0])
            if not selected:
                max_sim_sel = 0.0
            else:
                sims_with_sel = [cosine_sim_matrix[cid, s] for s in selected]
                max_sim_sel = max(sims_with_sel)
            mmr_score = mmr_lambda * rel - (1.0 - mmr_lambda) * max_sim_sel
            mmr_scores.append((cid, mmr_score))
        # pick best
        best_cid, _ = max(mmr_scores, key=lambda x: x[1])
        selected.append(best_cid)
        candidate_idxs.remove(best_cid)
    return selected

/tmp/ipython-input-1850771353.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['review_date'] = pd.to_datetime(df['review_date_creation'], errors='coerce')


In [221]:
# --- Example usage ---
review_idx = 10  # remplace par l'index interne de la critique active
cands = compute_combined_scores_for_review(review_idx, cosine_sim, df)
print("Top 5 par score brut:")
display(cands.head(5))

print("Top 5 après MMR:")
selected = mmr_rerank(review_idx, cands.head(50), cosine_sim, top_k=5, mmr_lambda=0.75)
display(df.loc[selected, ['review_title', 'review_hits', 'review_date']])

Top 5 par score brut:


,candidate_idx,text_sim,text_sim_norm,pop_norm,recency_norm,rating_norm,final_score
0,2,0.359470,0.679735,0.882956,1.468712e-05,1.000000,0.634434
1,0,0.271735,0.635867,1.000000,7.006073e-08,1.000000,0.631520
2,1,0.264610,0.632305,0.933609,8.757670e-06,0.777778,0.604995
3,35,0.437792,0.718896,0.632668,1.005263e-06,0.888889,0.602316
4,4,0.293465,0.646733,0.826503,5.509155e-06,0.888889,0.597785


Top 5 après MMR:


,review_title,review_hits,review_date
2,Sons of Anarchy.,12672.0,2015-07-19 02:55:00
0,Analyse du film et du livre,35817.0,2010-12-03 00:00:00
4,Il est temps d'enfreindre les deux premières r...,7677.0,2014-08-27 03:18:00
1,La consommation identitaire,19867.0,2015-01-28 09:33:00
12,Singe de l'espace.,3040.0,2011-04-11 00:00:00


#  Architecture (IA)

**Client :** UI qui demande recommandations pour une critique donnée.

**API Gateway :** routage + rate limiting + auth.

**Critique API :** lecture / retrieval de la critique; appelle Recommender Service.

**Recommender Service :**

Candidate generation (filtrer même film d’abord — exigence du sujet).

Scoring (textual similarity + signals: hits, likes, rating, recency, author similarity).

Re-ranking (MMR pour diversité, règles business).

Feature Store / DB : Postgres pour données relationnelles (reviews metadata). Redis pour cache top-N pré-calculés.

Vector Store / ANN (FAISS) : stocke embeddings SBERT pour recherche rapide en embeddings.

**Elasticsearch :** recherche full-text, facettes, filtres (utile si on veut recherches complexes).

**Batch infra :** calcule embeddings offline, pré-calc top-N, collecte signaux, updates nightly.

In [ ]:
                 +----------------------+
                 |  Client Web / Mobile |
                 +----------+-----------+
                            |
                            | HTTPS (REST / GraphQL)
                            v
                 +----------+-----------+
                 |  API Gateway / LB    |
                 +----+---------+-------+
                      |         |
         +------------+         +-----------------+
         |                                         |
+--------v-------+                         +-------v--------+
| Critique API   |                         | Recommender    |
| (FastAPI)      |                         | Service        |
| - GET /review/ |                         | - Candidate gen|
|   {id}         |                         | - Scoring      |
+--------+-------+                         | - Re-ranking   |
         |                                 +---+------------+
         |                                     |
         |                                     v
         |                           +---------+----------+
         |                           | Feature Store / DB |
         |                           | (Postgres / Redis) |
         |                           +---------+----------+
         |                                     |
         |                                     v
         |                          +----------+-----------+
         |                          | Vector Store / ANN   |
         |                          | (FAISS / Milvus)     |
         |                          +----------+-----------+
         |                                     |
         |                                     v
         |                          +----------+-----------+
         +------------------------->| Search / Index (ES)  |
                                    +----------+-----------+
                                               |
                                 Batch infra (offline recompute)
                                  Airflow/Celery -> compute embeddings,
                                  top-N precomputation -> cache (Redis)

# Note explicative

**1) Données**

Pour chaque review stocker au minimum : *review_id (unique); film_id / film_title; review_title; review_content; review_hits (affichages / vues); likes / upvotes / helpful_count; rating (note donnée par l’utilisateur, ex 0–10); user_id (auteur); review_date (timestamp); language; tags / genres; embedding_id (référence vers vector store); precomputed_topk (table stockant top-N déjà calculés).*

Ces colonnes servent pour : filtrage (same film), pondération business (popularité, fraîcheur), personnalisation (historique utilisateur), offline evaluation.

**2) Candidate generation & ranking — stratégie complète**

**Étapes (runtime)**

**Input :** *review_id* en cours de lecture.

**Filtrer :** candidates = toutes les reviews où *film_id* == film_of(*review_id*) (sauf la review elle-même).

**Si dataset film petit (< 100 reviews) :** on score tout (brut). Sinon on fait pré-filtre (ex. top 1000 par recency/popularité).

**Feature extraction :**

text_sim = similarité textuelle (TF-IDF cosine ou SBERT embedding cosine).

popularity = f(hits, likes) (ex. log(1+hits) ou normalisé).

recency = decay function sur review_date (ex. exp(-lambda * days_since)).

author_similarity (optionnel).

**Score final (linear blending) :**
score = w_text * text_sim_norm
      + w_pop  * pop_norm
      + w_rec  * recency_norm
      + w_rating * rating_norm

**Diversity / business rules :** appliquer MMR pour éviter redondance; blacklist (spam, deleted), min_length checks.

**Cache top-K pour reviews très consultés (Redis).**

**Return top-K (id, title, snippet, score).**

**Pourquoi cet ordre ?**

Exigence du sujet : même film → huge dimensionality drop (moins d’items à scorer). Candidate gen simple.

TF-IDF est rapide, SBERT donne sémantique — on peut utiliser TF-IDF pour MVP et SBERT pour upgrade.

Popularity/Recency improves UX : les meilleures/les plus récentes critiques ressortent.

**3) Choix technologiques & justification**

**TF-IDF (scikit-learn) :** MVP — rapide, pas besoin de GPU, facilement interprétable. Bon pour matching basé sur mots-clés (bagarre, combat).

**SBERT :** upgrade sémantique — paraphrases, meilleures recommandations mais plus coûteux.

**Postgres :** stockage principal (transactions, métadonnées).

**Elasticsearch :** recherche full text / filtres rapides (facultatif si petit produit).

**FAISS / Milvus :** vector search à l’échelle.

**Redis :** cache top-N pour latence ultra-basse.

**Kubernetes + FastAPI + Celery + Airflow :** production robuste/pipelines batch + realtime.